In [1]:
seed=1

In [2]:
import os

os.listdir('..')

In [3]:
from nasbench import api

nasbench_path = '../../../nasbench/nasbench_only108.tfrecord'
nb = api.NASBench(nasbench_path)

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 33 seconds


In [4]:
import torch
from info_nas.datasets.arch2vec_dataset import get_labeled_unlabeled_datasets

torch.backends.cudnn.benchmark = True
device = torch.device('cuda')

labeled, unlabeled = get_labeled_unlabeled_datasets(nb, percent_labeled=0.00001, device=device,
                                                    seed=seed)

Loading nasbench dataset (arch2vec) from ../data/nb_dataset.json

--- Preparing CIFAR10 Data ---
Files already downloaded and verified
Files already downloaded and verified
--- CIFAR10 Data Prepared ---

Split the dataset (percent labeled = 1e-05) - 4/381261 labeled networks chosen from the train set, 1/42363 labeled networks chosen from the validation set.
Processing labeled nets for the training set...
Loading labeled dataset from ../data/train_labeled.pt.
Processing labeled nets for the validation set...
Loading labeled dataset from ../data/valid_labeled.pt.


## Test dataset shapes

In [5]:
labeled.keys()

dict_keys(['train_io', 'train_net', 'valid_io', 'valid_net'])

In [6]:
[l.shape for l in labeled['train_io']]

[(4000,), torch.Size([4000, 128, 32, 32]), torch.Size([4000, 512, 8, 8])]

In [7]:
[l.shape for l in labeled['train_net']]

[torch.Size([4000, 7, 7]), torch.Size([4000, 7, 5])]

In [8]:
unlabeled.keys()

dict_keys(['train', 'n_train', 'val', 'n_val'])

In [9]:
unlabeled['train'][1][0].shape, unlabeled['train'][2][0].shape

(torch.Size([7, 7]), torch.Size([7, 5]))

In [10]:
len(unlabeled['train'][1])

381261

## Test model shapes

In [4]:
from arch2vec.extensions.get_nasbench101_model import get_arch2vec_model

model, opt = get_arch2vec_model()

In [11]:
from arch2vec.extensions.get_nasbench101_model import get_nasbench_datasets

nb_dataset = get_nasbench_datasets('../data/nb_dataset.json', batch_size=None, seed=1)

In [23]:
#model.train()
model.eval()

batch_adj, batch_ops = nb_dataset['train'][1][:32], nb_dataset['train'][2][:32]

mu, logvar = model._encoder(batch_ops, batch_adj)
z = model.reparameterize(mu, logvar)

In [25]:
print(f"mu shape: {mu.shape}, logvar shape: {logvar.shape}, z shape: {z.shape}")

mu shape: torch.Size([32, 7, 16]), logvar shape: torch.Size([32, 7, 16]), z shape: torch.Size([32, 7, 16])


In [34]:
import torch.nn as nn

m = nn.Sequential(
    nn.Flatten(),
    nn.Linear(z.shape[1] * z.shape[2], 5),
    nn.ReLU()
)
m(z).shape

torch.Size([32, 5])

In [43]:
# test unsqueeze and channels

conv = nn.Conv2d(32, 8, 1, padding=0)
conv(mu.unsqueeze(0)).shape

torch.Size([1, 8, 7, 16])

In [74]:
# repeat and concat
repeated = torch.Tensor([3]).repeat(mu.shape[0], mu.shape[1], 1)
print(repeated.shape)

torch.cat([mu, repeated], axis=-1).shape

torch.Size([32, 7, 1])


torch.Size([32, 7, 17])

## Extended models